In [1]:
import sys
import datetime
import time
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import os
import urllib
import tarfile
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from math import sin, cos, sqrt, atan2, radians

import glob

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

import geopandas as gpd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from matplotlib import cm
import haversine as hs
from shapely.geometry import MultiPoint, Point

from matplotlib.animation import FuncAnimation

### List of ROx: OH + HO2 + RO2

In [2]:
ROx = pd.read_excel('../data_input/ROx_GCv12.xlsx', sheet_name='031924_ROx_list', header=None)
ROx

,0
0,A3O2
1,ATO2
2,B3O2
3,BRO2
4,DHPCARP
5,DIBOO
6,ETO2
7,HC5OO
8,HO2
9,HPC52O2


In [3]:
ROx = ROx.iloc[:, 0].tolist()
print(f'Total # of ROx species is {len(ROx)}.')
sorted(ROx)
ROx

Total # of ROx species is 34.


['A3O2',
 'ATO2',
 'B3O2',
 'BRO2',
 'DHPCARP',
 'DIBOO',
 'ETO2',
 'HC5OO',
 'HO2',
 'HPC52O2',
 'IEPOXOO',
 'INO2',
 'ISNOOA',
 'ISNOOB',
 'ISNOHOO',
 'ISOPNBO2',
 'ISOPNDO2',
 'KO2',
 'LIMO2',
 'MAOPO2',
 'MO2',
 'MOBAOO',
 'MRO2',
 'OH',
 'OTHRO2',
 'PIO2',
 'PO2',
 'PRN1',
 'R4N1',
 'R4O2',
 'RIO2',
 'TRO2',
 'VRO2',
 'XRO2']

In [4]:
file = open("/projectsp/f_xj103_1/jqshen/Code.12.7.1.PHCHO_Original/KPP/Tropchem/Tropchem.eqn","r")
lines = file.readlines()

In [5]:
def func_radical_change(line, ROx):
    def process_term(st):
        for i in range(len(st)):
            if 'A' <= st[i] <= 'Z':
                break
        if i == 0:
            return 1., st
        else:
            return float(st[:i]), st[i:]
    line, _ = line.split(':')
    left, right = line.split('=')
    for c in [' ', '{+M}', '\n']:
        left = left.replace(c, '')
        right = right.replace(c, '')
    left = left.split('+')
    right = right.split('+')

    count = 0
    flag = 0
    for term in right:
        coef, spec = process_term(term)
        if spec in ROx:
            count += coef
    for term in left:
        coef, spec = process_term(term)
        if spec in ROx:
            count -= coef
            flag += 1
    if flag == 2:
        return count
    else:
        return None

In [6]:
flag = 0 # whether hv exists in eq, if hv exists, drop the whole equation
with open('../data_output/Tropchem_ROx_self_rxn.txt', 'w') as f:
    line_num = 394
    cur_eq = ''
    while line_num <= len(lines):
        if line_num < len(lines):
            if 'hv' in lines[line_num] or 'HET' in lines[line_num]:
                flag = 1
                cur_eq = ''
                line_num += 1
            else:
                if lines[line_num][0] != ' ': # start of a new equation
                    flag = 0
                    if len(cur_eq[:-1]):
                        count = func_radical_change(cur_eq, ROx)
                        if count is not None:
                            f.write(cur_eq[:-1] + f' --> {count:.3f}\n')
                    if lines[line_num][:2] == '//':
                        cur_eq = ''
                    else:
                        cur_eq = lines[line_num]
                else:
                    if not flag:
                        assert len(cur_eq)
                        assert lines[line_num][:2] != '//'
                        cur_eq = cur_eq[:-1] + lines[line_num]
        else:
            count = func_radical_change(cur_eq, ROx)
            if count is not None:
                f.write(cur_eq[:-1] + f' --> {count:.3f}\n')
        line_num += 1